In [26]:
import os
import openai
import docx2txt
import time
import pandas
import docx
import re
# Set up OpenAI API credentials
openai.api_key = "API KEY HERE"


# Function to apply GPT-3.5 model to a given text

In [27]:

# Define function to apply GPT-3.5 model to a given text
def generate_text(work, desc, rubrics):   
    description = desc
    rubrics=rubrics
    text = work
    instructor = """a language instructor and assess the work of students. I will provide you the assignment description/task given to students. 
        Then,  I will provide the student's work and the rubrics. 
        Your task will be to assign points and comments against each rubric first. 
        Then, Please highlight words, sentences, or phrases where the student made mistakes or violated rubrics.""" 
    prompt_aspect = f"""Act as {instructor} 
        Assignment description: 
        {description};    
        Rubrics:
        {rubrics}; 
        Student Work: 
        {text}"""

    try:
        result_score_turbo = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0.1, 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can assess student work and assign detailed feedback to students bu highlighting their mistakes or achievementso"},
            {"role": "user", "content": prompt_aspect},
            ]
        )
        score_turbo = result_score_turbo.choices[0].message.content.strip()
    except Exception as e:
        score_turbo = str(e) 
    result = score_turbo    
    # result = desc + rubrics+ text
    return result



# Functions to read files

In [28]:
# Define function to read text from a given Word file
def read_word_file(filename):

    # Open the Word file
    doc = docx.Document(filename)

    # Extract the text from the Word file
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)

    # Print the text from the Word file
    return "\n".join(text)

# Define function to read text from a given PDF file
def read_pdf_file(filename):
    with open(filename, 'rb') as f:
        reader = PyPDF2.PdfFileReader(f)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text



# Set essay id and Folder location

In [29]:
essay_id = 5
data_path = "asap-aes/"

# Read Rubrics and Descriptions

In [30]:
# desc = read_word_file(data_path+'Essay_Set_Descriptions/EssaySet'+str(essay_id)+'_Description.docx')
# rubrics = read_word_file(data_path+'Essay_Set_Descriptions/EssaySet'+str(essay_id)+'_Rubrics.docx')
prompt = read_word_file(data_path+'Essay_Set_Descriptions/Essay Set #'+str(essay_id)+'--ReadMeFirst.docx')
prompt= prompt.split('Rubric Guidelines', 1)
desc= prompt[0]
rubrics= prompt[1]
print(desc)

Essay Set #5
Source Essay
Narciso Rodriguez
from Home: The Blueprints of Our Lives
My parents, originally from Cuba, arrived in the United States in 1956. After living for a year in a furnished one-room apartment, twenty-one-year-old Rawedia Maria and twenty-seven-year-old Narciso Rodriguez, Sr., could afford to move into a modest, three-room apartment I would soon call home.
In 1961, I was born into this simple house, situated in a two-family, blond-brick building in the Ironbound section of Newark, New Jersey. Within its walls, my young parents created our traditional Cuban home, the very heart of which was the kitchen. My parents both shared cooking duties and unwittingly passed on to me their rich culinary skills and a love of cooking that is still with me today (and for which I am eternally grateful). Passionate Cuban music (which I adore to this day) filled the air, mixing with the aromas of the kitchen. Here, the innocence of childhood, the congregation of family and friends, an

In [31]:
print(rubrics)


Score Point 4: The response is a clear, complete, and accurate description of the mood created by the author. The response includes relevant and specific information from the memoir.
 
Score Point 3: The response is a mostly clear, complete, and accurate description of the mood created by the author. The response includes relevant but often general information from the memoir.
 
Score Point 2: The response is a partial description of the mood created by the author. The response includes limited information from the memoir and may include misinterpretations.
 
Score Point 1: The response is a minimal description of the mood created by the author. The response includes little or no information from the memoir and may include misinterpretations. 
OR 
The response relates minimally to the task.

Score Point 0: The response is incorrect or irrelevant or contains insufficient information to demonstrate comprehension.
Adjudication Rules
For the specific set of data, if there was a difference

# Call the process_files function

In [32]:

# Create a list of unique rubric names
unique_rubrics = ["Score", "Point"]
# Create a new dictionary for each unique rubric name to store the assessment results
rubric_results = {rubric: [] for rubric in unique_rubrics}
# df.head(5)
keywords = ["overall statement", "Comments", "Mistakes"]
results = {rubric: [] for rubric in keywords}
feedback= []


In [33]:
import pandas as pd
if(os.path.isfile(data_path+"results_gpt/EssaySet"+str(essay_id)+"_assessment_results_by_rubric.csv")):
    temp = pd.read_csv(data_path+"results_gpt/EssaySet"+str(essay_id)+"_assessment_results_by_rubric.csv")
else:
    temp=[]
print(len(temp))
# Load the data from a CSV file
df = pd.read_excel(data_path+"training_set_rel3.xlsx")
df = df[df['essay_set'] == essay_id]
df = df[len(temp):]
df = df.dropna(axis=1, how='all')
print(len(df))
df.head(2)


1805
0


""


In [34]:
jump=2
for i in range(0, len(df), jump):
    data = df[i:i+jump] 
    # Iterate over the rows in the data
    for index, row in data.iterrows():

        # Get the text to assess
        text_to_assess = row["essay"]    
        print(text_to_assess)
        # Apply the OpenAI model to the text to get the assessment result
        assessment_result = generate_text(work= text_to_assess, desc=desc, rubrics=rubrics)
        feedback.append(assessment_result)    
        print(assessment_result)
        # Extract the rubric values from the assessment result text
        for rubric in unique_rubrics:
            pattern = re.compile(rf"(?i){rubric}\D+(\d+(\.\d+)?)")
            match = pattern.search(assessment_result)
            print(match)
            if match:
                rubric_results[rubric].append(match.group(1))
            else:
                rubric_results[rubric].append("")
        for i in range(len(keywords)):
            keyword = keywords[i]
            pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
            match = pattern.search(assessment_result)
            if match:
                results[keyword].append(match.group(1).strip())
            else:
                results[keyword].append("")
        time.sleep(20)            
        print(rubric_results)

    for rubric in unique_rubrics:
        data["domain1_"+rubric+"_gpt"] = rubric_results[rubric]
    for keyword in keywords:
        data[keyword+"_gpt"] = results[keyword]
    data['Response_gpt'] = feedback
    # Save the data with the rubric columns to a new CSV file
    data.to_csv(data_path+"results_gpt/EssaySet"+str(essay_id)+"_assessment_results_by_rubric.csv", index = False, mode='a', header=False)
    # data.to_csv(data_path+"results_gpt/EssaySet"+str(essay_id)+"_assessment_results_by_rubric.csv", index = False)
    rubric_results = {rubric: [] for rubric in unique_rubrics}
    results = {rubric: [] for rubric in keywords}
    feedback=[]